In [ ]:
%pip install setuptools==70.3.0  # apparently setuptools < 71.0.0 is required for a required package of openml to install on here
%pip install openml
%pip install numpy
%pip install pandas
%pip install torch torchvision torchaudio  # cuda 12.4, cf. nvidia-smi shell command, cf. https://pytorch.org/get-started/locally/
%pip install -U scikit-learn
%pip install scipy
%pip install -U pymoo
%pip list

In [1]:
from openml import tasks

from classes import EAGGA

import os

In [2]:
oml_task_ids = [37, 43, 3903, 3904, 3913, 3918, 10093, 9946, 146819, 359955, 189922, 359962, 190392, 167120, 190137, 190410, 168350, 359975, 359972, 146820]
oml_tasks = tasks.get_tasks(oml_task_ids)

oml_datasets = [oml_task.get_dataset() for oml_task in oml_tasks]

# define positive classes
positive_classes = ['tested_positive', '1', True, True, 'yes', True, '2', '2', '1', '2', '1', True, '1', '1', '2', '1', '2', 'Anomaly', '1', '2']

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openml/tasks/functions.py:372: UserWarning: `download_data` will default to False starting in 0.16. Please set `download_data` explicitly to suppress this warning.
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openml/tasks/functions.py:380: UserWarning: `download_qualities` will default to False starting in 0.16. Please set `download_qualities` explicitly to suppress this warning.
  warnings.warn(


In [ ]:
hps = {
    'total_layers': (3, 10),
    'nodes_per_hidden_layer': (3, 20),
    'mu': 15,  # TODO: 100
    'lambda': 5,  # TODO: 10
    'holdout_train_size': 2/3,
    'cv_k': 5
}

batch_size = 64
patience = 10

secs_per_fold = 30
secs_total = 20 * 60

for (oml_dataset, class_positive) in zip(oml_datasets[:1], positive_classes[:1]):  # TODO: remove [:1]
    name = oml_dataset.name
    print(f'Dataset {name}')

    file_path = os.path.join('export', name)
    
    eagga = EAGGA(
        oml_dataset=oml_dataset,
        class_positive=class_positive,
        hps=hps,
        batch_size=batch_size,
        patience=patience,
        secs_per_fold=secs_per_fold,
        secs_total=secs_total,
        file_path=file_path
    )
    #eagga.load_population(6)
    eagga.run_eagga()